In [1]:
import ee 
import geemap
ee.Initialize()

# Export image collection 

In [2]:
point = ee.Geometry.Point(-99.2222, 46.7816)
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filterBounds(point)
    .filterDate('2008-01-01', '2018-01-01')
    .filter(ee.Filter.listContains("system:band_names", "N"))
)
   

In [3]:

collection.aggregate_array('system:index').getInfo()

['m_4609915_sw_14_1_20090818',
 'm_4609915_sw_14_1_20100629',
 'm_4609915_sw_14_1_20120714',
 'm_4609915_sw_14_1_20140901',
 'm_4609915_sw_14_1_20150926',
 'm_4609915_sw_14_h_20160704',
 'm_4609915_sw_14_h_20170703']

In [4]:
collection

Name,Description
R,Red
G,Green
B,Blue
N,Near infrared


In [5]:
out_dir='naip'
geemap.ee_export_image_collection(collection, out_dir=out_dir, scale= 10)

Total number of images: 7

Exporting 1/7: naip\m_4609915_sw_14_1_20090818.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\naip\m_4609915_sw_14_1_20090818.tif


Exporting 2/7: naip\m_4609915_sw_14_1_20100629.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\naip\m_4609915_sw_14_1_20100629.tif


Exporting 3/7: naip\m_4609915_sw_14_1_20120714.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\naip\m_4609915_sw_14_1_20120714.tif


Exporting 4/7: naip\m_4609915_sw_14_1_20140901.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\naip\m_4609915_sw_14_1_20140901.tif


Exporting 5/7: naip\m_4609915_sw_14_1_20150926.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\naip\m_4609915_sw_14_1_20150926.tif


Exporting 6/7: naip\m_4609915_sw_

### To drive

In [6]:
geemap.ee_export_image_collection_to_drive(collection, folder='export', scale=10)

In [3]:
geemap.image_dates(collection).sort()

In [4]:
collection

Name,Description
R,Red
G,Green
B,Blue
N,Near infrared


In [5]:
m= geemap.Map()
m.addLayer(collection.first(), {}, 'First image')
m.centerObject(point, 12)
m

Map(center=[46.7816, -99.22220000000002], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [6]:
m.user_roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -99.321213,
              46.618318
            ],
            [
              -99.321213,
              46.714915
            ],
            [
              -99.026642,
              46.714915
            ],
            [
              -99.026642,
              46.618318
            ],
            [
              -99.321213,
              46.618318
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [7]:
ts= geemap.naip_timeseries(roi=m.user_roi, start_year=2009, end_year=2010, RGBN=True)

### Create a time series for satellite images
`geemap.create_timeseries()` or `geemap.naip_timeseries()`

In [8]:
ts

In [9]:
m= geemap.Map()
image= ts.first()
m.addLayer(image, {}, 'First image')
m.center_object(ts, 12)
m

Map(center=[46.66663361403033, -99.17392750000012], controls=(WidgetControl(options=['position', 'transparent_…

In [11]:
roi = ee.Geometry.Point([-122.44, 37.75])
collection = (
    ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')
    .filterBounds(roi)
    .sort("CLOUD_COVER")
    .limit(10)
)

image = collection.first()

Map = geemap.Map()

vis_params = {
    'bands': ['B5', 'B4', 'B3'],
    'min': 0,
    'max': 0.3,
    'gamma': [0.95, 1.1, 1],
}

Map.addLayer(image, vis_params, "LANDSAT 8")
Map.setCenter(-122.44, 37.75, 8)
Map

Map(center=[37.75, -122.44], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

## Exporting image thumbnails

In [12]:
out_img = 'landsat.jpg'
region = ee.Geometry.BBox(-122.5955, 37.5339, -122.0982, 37.8252)
geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=1000, region=region)

In [13]:
geemap.show_image(out_img)

Output()

### Downloading several thumbnails images at onece

`geemap.get_image_collection_thumbnails()`


Download thumbnails one by one

```python
geemap.get_image_collection_thumbnails(collection, out_dir, vis_params, dimensions=512, region=region)

```

In [14]:
out_dir= 'thumbnails'
geemap.get_image_collection_thumbnails(collection, out_dir, vis_params, dimensions=1000, region=region)

Total number of images: 10



### Make GIF file based on export thumbnails

In [15]:
geemap.make_gif('thumbnails', 'landsat.gif',fps=3)

# Download feature collections (vector data)

In [16]:
Map = geemap.Map()
fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(
    ee.Filter.eq('country_co', 'KV')
)

Map.addLayer(fc, {}, "Europe")
Map.centerObject(fc, 10)
Map

Map(center=[42.58077111456963, 20.889411432780253], controls=(WidgetControl(options=['position', 'transparent_…

In [18]:
geemap.ee_to_shp(fc, filename='kv.shp')

### exporting functions ```geemap.ee_to_vector(fc, filname='.shp,.geojson,.csv')```

In [19]:
geemap.ee_export_vector(fc, filename='kv.geojson')

Generating URL ...
Please wait ...
Data downloaded to c:\Users\zyang\spatial_data_management\notes\kv.geojson


In [20]:
gdf= geemap.ee_to_gdf(fc)
gdf

,geometry,abbreviati,country_co,country_na,wld_rgn
0,"POLYGON ((20.01428 42.74082, 20.01628 42.73519...",Kos.,KV,Kosovo,Europe


In [21]:
df= geemap.ee_to_df(fc)
df

,abbreviati,country_co,country_na,wld_rgn
0,Kos.,KV,Kosovo,Europe


In [22]:
gdf.explore()

# Exporting maps

In [23]:
m= geemap.Map()
image= ee.Image('USGS/SRTMGL1_003')
vis_params= {
    'min': 0,
    'max': 5000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
}
m.addLayer(image, vis_params, 'SRTM DEM')
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [24]:
m.to_html('dem.html', title='Earth Engine Map', width='100%', height='800px')
# downside: only works around 24 hours, the layer is temporary.
# Web App will works all the time